In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import nltk
import requests

In [0]:
nltk.download('punkt')
nltk.download('stopwords')

In [0]:
input = """how to define function in python""" 

from nltk.tokenize import word_tokenize
tokenized_word=word_tokenize(input)

from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
filtered_sent=[]
for w in tokenized_word:
    if w not in stop_words:
        filtered_sent.append(w)

In [0]:
URL = 'https://api.stackexchange.com/2.2/questions?order=asc&sort=activity&tagged=python%3Bfunction%3Bdefinition&site=stackoverflow'

r = requests.get(url = URL)
data = r.json()
messages = []
for item in data['items']:
  messages.append(item['title'])

messages.append(input) 

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 

embed = hub.Module(module_url)

In [0]:
similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)
with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  message_embeddings = session.run(similarity_message_encodings, feed_dict={similarity_input_placeholder: messages})
  per = np.inner(message_embeddings, message_embeddings[-1:])


In [0]:
print(per)